# Data Cleaning

In [1]:
# Import libraries 
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# Melbourne Housing Dataset 
df = pd.read_csv("/Users/adapanjali/Desktop/Courses/Kaggle/melb_data.csv")

In [3]:
# Separating target variable from data

# Save target to another varible 
y = df.Price
# Drop target from the variable 
X = df.drop(['Price'], axis=1)

In [4]:
# Divide data into training and validation sets 
X_train_full, X_test_full, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [5]:
# Drop columns with missing values (simple method)
cols_with_missing = [col for col in X_train_full.columns 
                                if X_train_full[col].isnull().any()]
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_test_full.drop(cols_with_missing, axis=1, inplace=True)

In [6]:
# Selecting categorical columns with relatively low cardinality 
low_cardinality_cols = [col for col in X_train_full.columns
                                if X_train_full[col].nunique() < 10 and X_train_full[col].dtype == "object"]

# Selecting all the numerical columns 
num_cols = [col for col in X_train_full.columns
                    if X_train_full[col].dtype in ["int64", "float64"]]

In [7]:
# Keeping only columns with low cardinality values and the numerical columns 
cols = low_cardinality_cols + num_cols # List append 
X_train = X_train_full[cols]
X_test = X_test_full[cols]

In [8]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0


In [9]:
# Get a list of all the categorical variables
obj_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

print("Categorical Variables:")
print(obj_cols)

Categorical Variables:
['Type', 'Method', 'Regionname']


In [10]:
# Function to Measure Quality of Different Approaches 

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_test, y_train, y_test):
    model = RandomForestRegressor(n_estimators= 100, random_state= 0)
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)

    return round(mean_absolute_error(y_test, prediction), 2)

# Working with Categorical Variables

## 1. Dropping categorical variables

In [11]:
drop_X_train = X_train.select_dtypes(exclude= ["object"]).copy()
drop_X_test = X_test.select_dtypes(exclude= ["object"]).copy()

In [12]:
print("MAE from Approach 1 (Drop Categorical Variables):")
print(score_dataset(drop_X_train, drop_X_test, y_train, y_test))

MAE from Approach 1 (Drop Categorical Variables):
175703.48


## 2. Ordinal Encoding

In [13]:
from sklearn.preprocessing import OrdinalEncoder

In [14]:
label_X_train = X_train.copy()
label_X_test = X_test.copy()

In [15]:
ordinal_encoder = OrdinalEncoder()
label_X_train[obj_cols] = ordinal_encoder.fit_transform(X_train[obj_cols])
label_X_test[obj_cols] = ordinal_encoder.transform(X_test[obj_cols])

In [16]:
print("MAE from Approach 2 (Ordinal Encoding):") 
print(score_dataset(label_X_train, label_X_test, y_train, y_test))

MAE from Approach 2 (Ordinal Encoding):
165936.41


## 3. One Hot Encoding 

In [17]:
from sklearn.preprocessing import OneHotEncoder

In [18]:
# Apply one-hot encoder to each column with categorical data
# Only the object columns are one-hot encoded, we turn this matrix into a dataframe 

oh_encoder = OneHotEncoder(handle_unknown= "ignore", sparse= False)
oh_cols_train = pd.DataFrame(oh_encoder.fit_transform(X_train[obj_cols]))
oh_cols_test = pd.DataFrame(oh_encoder.transform(X_test[obj_cols]))

In [19]:
# One-hot encoding removed index; put it back
oh_cols_train.index = X_train.index
oh_cols_test.index = X_test.index

In [20]:
num_X_train = X_train.drop(obj_cols, axis= 1)
num_X_test = X_test.drop(obj_cols, axis= 1)

In [21]:
oh_X_train = pd.concat([num_X_train, oh_cols_train], axis= 1)
oh_X_test = pd.concat([num_X_test, oh_cols_test], axis= 1)

In [22]:
print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(oh_X_train, oh_X_test, y_train, y_test))

MAE from Approach 3 (One-Hot Encoding):


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


166089.49


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
